# Выявление особенностей словоупотребления в текстах Л. Н. Толстого методами дистрибутивной семантики

** TO DO **

- обучить сегментатор поточнее
- проверить его на "медленной" и "быстрой" версиях (по количеству предложений)
- проверить теги для дневников и писем
- почистить мета-информацию из дневников и писем
- "скормить" майстему дореволюционный (вроде ест)

In [78]:
import os
import re
from lxml import etree
from pymystem3 import Mystem
import string
import nltk
from langdetect import detect
import langid
from datetime import datetime
mystem = Mystem()

## Сегментатор

In [2]:
language_punkt_vars = nltk.tokenize.punkt.PunktLanguageVars
language_punkt_vars.sent_end_chars=('.','?','!','»','…')
with open('.\\data\\traindata.txt', 'r', encoding='utf-8') as f:
    traindata = f.read()
trainer = nltk.tokenize.punkt.PunktTrainer(traindata, language_punkt_vars)
trainer.INCLUDE_ALL_COLLOCS = True 
trainer.INCLUDE_ABBREV_COLLOCS = True
params = trainer.get_params()
sbd = nltk.tokenize.punkt.PunktSentenceTokenizer(params)

  Abbreviation: [4.0590] г
  Abbreviation: [1.0986] руб
  Abbreviation: [1.4932] ст
  Abbreviation: [0.6063] матф
  Abbreviation: [4.0590] н
  Abbreviation: [4.0590] п
  Abbreviation: [5.9728] т.п
  Abbreviation: [19.4117] см
  Abbreviation: [0.5493] стр
  Abbreviation: [1.4932] ул
  Abbreviation: [5.4932] тыс
  Abbreviation: [4.0590] м
0.00038153376573826786 0.007722007722007722 0.0 5242 259 2 2
0.00038153376573826786 0.007722007722007722 0.0 5242 259 2 2
0.003052270125906143 0.007722007722007722 0.0028095524784266506 5242 259 16 2
0.009538344143456695 0.03861003861003861 0.008027292795504716 5242 259 50 10
0.003052270125906143 0.007722007722007722 0.0028095524784266506 5242 259 16 2
0.0011446012972148034 0.007722007722007722 0.0008027292795504716 5242 259 6 2
0.0036245707745135446 0.007722007722007722 0.0034115994380895043 5242 259 19 2
0.0007630675314765357 0.007722007722007722 0.0004013646397752358 5242 259 4 2
0.0015261350629530714 0.023166023166023165 0.0004013646397752358 5242 2

## Данные

На основе 90-томного собрания сочинений

In [3]:
tei_ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

In [4]:
string.punctuation = string.punctuation.replace('-','')
string.punctuation

'!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

## Препроцессинг

In [5]:
def get_one_writing(filename):
    root = etree.parse(filename)#.getroot()
    for i in tags_for_deletion:
        for j in root.findall(i, tei_ns):
            parent = j.find("..")
            parent.remove(j)
    for i in tags_to_strip:
        etree.strip_tags(root, i)
    return root.find(".//tei:text", tei_ns).text.strip()

In [86]:
def preprocessing(text, f):
    #text = re.sub('[0-9—A-Za-zÉéÂâÊêÎîÔôÛûÀàÈèÙùËëÏïÜüŸÿÇç«»’§°¼½×áä˃<]+','', text)
    text = re.sub('[\n\s]+',' ', text)
    print (len(sbd.sentences_from_text(text, realign_boundaries=True)), end='\t')
    text = ' $$ '.join(sbd.sentences_from_text(text, realign_boundaries=True))
    text = text.replace('[','').replace(']','')
    text = ''.join(mystem.lemmatize(text.strip()))
    text = re.sub('[^а-яА-Я\-$ ]','', text)
    #text = re.sub('[0-9—«»’§°¼½×˃<…“„]+','', text)
    text = re.sub('[-]{2,}','', text)
    text = text.split(' $$ ')
    for i in text:
        i2 = i.translate(i.maketrans('','',string.punctuation))
        i2 = re.sub('[\n\s]+',' ', i2)
        lid = langid.classify(i2)[0]
        if lid not in ['fr','la','el'] and len(i2)>25:
            f.write(i2.strip()+'\n')
            pass

In [48]:
def universal_preprocessing(path, 
                            tags_for_deletion, 
                            tags_to_strip, 
                            filename):
    tags_for_deletion = tags_for_deletion
    tags_to_strip = tags_to_strip
    mystem = Mystem()
    start_gen = datetime.now()
    with open (filename, 'w', encoding='utf-8') as f:
        for root, dirs, files in os.walk (path):
                for fl in files :
                    start = datetime.now() 
                    preprocessing(get_one_writing(root+'\\'+fl), f)
                    print ('{}\t{}\t{}'.format(os.path.getsize(root+'\\'+fl)//1024, 
                                              datetime.now()-start,
                                              fl))
    print ('\n\ntotal time:\t\t {}'.format(datetime.now()-start_gen))
    

### Произведения

** Плохие теги: **

- note
- add
- sic
- orig
- head
- del
- figure
- table


** Ненужные теги: **

- pb
- hi
- corr
- reg
- choice
- div
- p

In [ ]:
preprocessing()

In [32]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb']

In [87]:
path = '.\\data\\Final\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'writings.txt')

938	125	0:00:10.254526	Abrégé de l’Evangile 24.xml
21	8	0:00:02.685905	AMOUR DE LA PATRIE 1.xml
70	37	0:00:04.564750	Carthago delenda est 39.xml
87	32	0:00:04.358475	Carthago delenda est. Черновое 39.xml
322	95	0:00:05.436625	Letter to a Hindoo (Письмо к индусу) 37.xml
44	13	0:00:02.058494	NOTES SUR LE SECOND CHAPITRE DES «CARACTÈRES» DE LA BRUYÈRE 1.xml
28	13	0:00:02.138017	The hostelry (Гостиница) 37.xml
19	19	0:00:02.371734	[Carthago delenda est] («Жизнь, та форма жизни»...) 27.xml
9	5	0:00:01.858721	[LE PRÉSENT, LE PASSÉ ET LE FUTUR] 1.xml
9	9	0:00:01.944496	[«Давно позабыл я о счастьи ...»] 1.xml
49	29	0:00:02.185548	[«Дядинька Жданов и кавалер Чернов»] 3.xml
60	42	0:00:02.512782	[«Ежели бы человек не желал…»] 1.xml
18	12	0:00:01.911855	[«Ей, Марьяна, брось работу!..»] 1.xml
37	22	0:00:01.919879	[«Знание музыки - субъективное и объективное...»] 1.xml
729	330	0:00:06.854363	[«Начала» романа «Сто лет»] 17.xml
372	203	0:00:04.548956	[«Начала» романа времен Петра I. Азовские походы] 1

In [73]:
s = 'Je conçois que cest ainsi que tu las voulu'
langid.classify(s)[0]

'pt'

### Письма

opener
note
pb
closer

In [8]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table',
                      './/tei:opener', './/tei:closer']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb','{http://www.tei-c.org/ns/1.0}body']

In [9]:
root = etree.parse(".\\data\\LETTERS\\Volume_59\\letter101.xml")

XMLSyntaxError: ID note1 already defined, line 95, column 931 (./data/LETTERS/Volume_59/letter101.xml, line 95)

In [16]:
def universal_preprocessing_try(path, 
                            tags_for_deletion, 
                            tags_to_strip, 
                            filename):
    tags_for_deletion = tags_for_deletion
    tags_to_strip = tags_to_strip
    mystem = Mystem()
    start_gen = datetime.now()
    bad = []
    with open (filename, 'w', encoding='utf-8') as f:
        for root, dirs, files in os.walk (path):
                for fl in files :
                    start = datetime.now() 
                    try:
                        preprocessing(get_one_writing(root+'\\'+fl), f)
                        #print ('{}\t{}\t{}'.format(os.path.getsize(root+'\\'+fl)//1024, 
                        #                          datetime.now()-start,
                        #                          fl))
                    except:
                        bad.append(root+'\\'+fl)
    print ('\n\ntotal time:\t\t {}'.format(datetime.now()-start_gen))
    print('\n'.join(bad))
    

In [17]:
path = '.\\data\\LETTERS\\'
universal_preprocessing_try(path, tags_for_deletion, tags_to_strip, 'letters.txt')

2	5	2	3	1	12	2	4	3	3	3	4	1	4	1	3	14	6	4	2	6	7	2	3	11	2	6	3	4	5	6	0	1	5	7	10	7	5	1	13	6	3	1	1	1	6	10	3	3	8	18	7	7	6	1	2	5	4	11	4	2	1	2	2	1	5	6	2	2	4	10	9	4	2	4	16	9	15	16	2	2	7	1	2	6	5	18	2	9	7	11	6	11	3	1	1	2	2	2	1	2	22	8	11	15	2	10	2	4	6	2	3	4	7	1	3	1	2	17	2	11	1	23	15	8	7	4	4	6	4	10	14	2	4	1	3	2	4	5	4	5	9	2	4	2	4	11	17	21	6	6	5	3	1	5	4	8	2	1	6	1	1	1	4	5	1	1	5	3	14	3	2	1	1	4	9	5	3	2	1	3	4	4	5	7	3	1	3	3	2	1	3	2	3	3	5	6	7	7	2	7	3	10	1	9	2	1	4	4	3	4	1	1	1	12	1	3	5	2	0	1	0	2	14	13	1	13	1	2	1	6	3	8	2	4	2	1	7	4	5	3	7	4	1	2	13	6	11	13	6	2	1	4	10	9	1	7	11	5	4	4	2	6	2	1	2	6	6	7	2	1	1	1	7	1	3	1	10	2	7	5	4	4	1	4	1	16	1	2	1	3	4	1	10	11	1	6	3	6	1	8	3	12	3	1	9	8	6	4	3	1	13	3	8	4	4	22	2	2	2	3	4	1	21	10	2	9	2	2	2	5	4	2	8	2	4	0	7	10	2	4	9	3	4	8	11	3	2	3	2	3	5	5	7	5	19	3	7	4	7	2	3	2	0	10	20	5	2	5	5	5	4	3	0	4	5	2	2	9	17	2	7	2	0	2	0	3	0	4	4	2	4	2	2	6	2	6	5	12	0	3	5	2	5	13	2	4	0	2	3	4	0	5	2	2	14	3	0	8	6	7	6	2	0	4	0	7	3	7	10	10	2	6	0	6	2	2	10	12	12	21	0	8	6	8	5	5	7	7	5	2	13	3	2	7	10	4	2	7	18	0	6	4	3	6	0	2	

### Дневники

opener
note



div
p

In [21]:
tags_for_deletion  = ['.//tei:note','.//tei:add','.//tei:sic','.//tei:orig',
                      './/tei:head','.//tei:del','.//tei:figure','.//tei:table', './/tei:opener']

tags_to_strip = ['{http://www.tei-c.org/ns/1.0}pb', '{http://www.tei-c.org/ns/1.0}hi', '{http://www.tei-c.org/ns/1.0}corr',
                '{http://www.tei-c.org/ns/1.0}reg', '{http://www.tei-c.org/ns/1.0}choice', '{http://www.tei-c.org/ns/1.0}p',
                '{http://www.tei-c.org/ns/1.0}div', '{http://www.tei-c.org/ns/1.0}lb','{http://www.tei-c.org/ns/1.0}body']

In [22]:
path = '.\\data\\diaries\\'
universal_preprocessing(path, tags_for_deletion, tags_to_strip, 'diaries.txt')

845	981	0:00:10.897919	46.xml
524	698	0:00:04.896199	47.xml
2675	268	0:00:17.522655	48.xml
4700	425	0:00:28.088938	49.xml
6204	778	0:00:38.901313	50.xml
3504	441	0:00:23.632458	51.xml
3829	612	0:00:27.714655	52.xml
4721	897	0:00:33.016014	53.xml
1583	752	0:00:13.058729	54.xml
2210	1117	0:00:20.002205	55.xml
2576	639	0:00:20.198545	56.xml
2834	733	0:00:21.618889	57.xml
2452	568	0:00:17.266406	58.xml


total time:		 {} 0:04:36.820934


In [28]:
path = '.\\data\\diaries\\'
for root, dirs, files in os.walk (path):
        for fl in files :
            print(root+'\\'+fl)

.\data\diaries\\46.xml
.\data\diaries\\47.xml
.\data\diaries\\48.xml
.\data\diaries\\49.xml
.\data\diaries\\50.xml
.\data\diaries\\51.xml
.\data\diaries\\52.xml
.\data\diaries\\53.xml
.\data\diaries\\54.xml
.\data\diaries\\55.xml
.\data\diaries\\56.xml
.\data\diaries\\57.xml
.\data\diaries\\58.xml


In [29]:
get_one_writing(".\data\diaries\\46.xml")

''

In [15]:
filename = ".\data\diaries\\46.xml"
root = etree.parse(filename)#.getroot()
for i in tags_for_deletion:
    for j in root.findall(i, tei_ns):
        parent = j.find("..")
        parent.remove(j)
for i in tags_to_strip:
    etree.strip_tags(root, i)
root.find(".//tei:text", tei_ns).text.strip()
#etree.tostring(root)

"Казань]. 17 Мар. Вотъ уже шесть дней, какъ я поступилъ въ клинику, и вотъ шесть дней, какъ я почти доволенъ собою. Les petites causes produisent de grands effets.Гаонарею, понимается, отъ того, отъ чего она обыкновенно получается; и это пустое обстоятельство дало мнѣ толчокъ, отъ котораго я сталъ на ту ступень, на которой я уже давно поставилъ ногу; но никакъ не могъ перевалить туловище (отъ того, должно быть, что не обдумавши поставилъ лѣвую ногу вмѣсто правой). Здѣсь я совершенно одинъ, мнѣ никто не мѣшаетъ, здѣсь у меня нѣтъ услуги, мнѣ никто не помогаетъ — слѣдовательно на разсудокъ и память ничто постороннее не имѣетъ вліянія, и дѣятельность моя необходимо должна развиваться. Главная же польза состоитъ въ томъ, что я ясно усмотрѣлъ, что безпорядочная жизнь, которую большая часть свѣтскихъ людей принимаютъ зa слѣдствіе молодости, есть ничто иное, какъ слѣдствіе ранняго разврата души.очки,3 4которые показывали ему все въ превратномъ видѣ, и какъ уяснится взглядъ его на вѣщи, такъ ч

## Обработка текста

# Try

In [18]:
import gensim

C:\Users\Glaz\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [88]:
sentences = gensim.models.word2vec.LineSentence('writings.txt')

In [82]:
#sentences = [['я', 'иду'], ['я', 'иду']]

In [94]:
#start = datetime.now() 
%time
model = gensim.models.Word2Vec(sentences, min_count=2, sorted_vocab=1)
#print(datetime.now()-start)

Wall time: 0 ns


In [95]:
print(model)

Word2Vec(vocab=29551, size=100, alpha=0.025)


In [ ]:
print(model.wv.vocab.keys())